In [1]:
import pandas as pd
import numpy as np

# Read taxonomy table

In [2]:
df_tax = pd.read_csv('taxonomy.tsv', sep='\t')
taxonomy = []
for idx in df_tax.index:
    taxon = df_tax.loc[idx,'Taxon']
    taxon = taxon.split(';')
    res = [df_tax.loc[idx,'Feature ID']]
    lowest_classified_taxon = 'unclassified'
    for i in np.arange(7):
        if i < len(taxon):
            res.append(taxon[i])
            if 'unclassified' not in taxon[i] and 'uncultured' not in taxon[i] and 'unidentified' not in taxon[i]:
                lowest_classified_taxon = taxon[i]
        else:
            res.append('unclassified')
    res.append(lowest_classified_taxon)
    taxonomy.append(res)
    
df_tax = pd.merge(df_tax, pd.DataFrame(taxonomy, columns=['Feature ID','Kingdom','Phylum','Class','Order','Family','Genus','Species','LowestClassifiedTaxon']), left_on='Feature ID', right_on='Feature ID', how='left')
df_tax = df_tax[['Feature ID','Confidence','Kingdom','Phylum','Class','Order','Family','Genus','Species','LowestClassifiedTaxon']]
df_tax = df_tax.rename({'Feature ID':'ASV'}, axis=1).set_index('ASV')

# remove genus tag in species
df_tax['Species'] = [y.replace(x+'-','') for x,y in zip(df_tax['Genus'], df_tax['Species'])]
#df_tax['LowestTaxon'] = [y.replace(x+'-','') for x,y in zip(df_tax['Genus'], df_tax['LowestTaxon'])]

# select only bacteria
# df_tax = df_tax[df_tax.Kingdom=='Bacteria']

df_tax.head()

,Confidence,Kingdom,Phylum,Class,Order,Family,Genus,Species,LowestClassifiedTaxon
ASV,,,,,,,,,
d272bf25781448dde9031a24679a9012,0.964328,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,acidifaciens,Bacteroides-acidifaciens
d33d84f90d2924040cf2eb66c7abe2fc,0.929414,Bacteria,Firmicutes,Bacilli,Lactobacillales,Lactobacillaceae,Lactobacillus,murinus,Lactobacillus-murinus
ec098ad12ef2923b449a01762462578b,1.000000,Bacteria,Firmicutes,Bacilli,Lactobacillales,Streptococcaceae,Lactococcus,unclassified,Lactococcus
cb7b937fdccf682f018b57aa6b9d92b2,1.000000,Bacteria,Actinobacteriota,Actinobacteria,Bifidobacteriales,Bifidobacteriaceae,Bifidobacterium,unclassified,Bifidobacterium
b6c1a8ba78f65c4a4e96627ccc53ac86,1.000000,Bacteria,Firmicutes,Bacilli,Lactobacillales,Lactobacillaceae,Lactobacillus,unclassified,Lactobacillus


# Prepare reIf_longitudinalve abundance table

In [3]:
df_count = pd.read_csv('otu.txt', sep='\t')
df_count = df_count.rename({'#OTU ID':'ASV'}, axis=1).set_index('ASV')
df_count = df_count.div(df_count.sum(axis=0), axis=1)
df_count = df_count[(df_count.T != 0).any()]

# merge tax and absolute abundance
df_count = pd.merge(df_tax[['LowestClassifiedTaxon']], df_count, left_index=True, right_index=True, how='inner').set_index('LowestClassifiedTaxon', drop=True)

# merge index (row sum for the same index)
df_count = df_count.groupby(df_count.index).sum()

df_count.head()

,sample1,sample10,sample100,sample101,sample102,sample103,sample104,sample105,sample106,sample107,...,sample90,sample91,sample92,sample93,sample94,sample95,sample96,sample97,sample98,sample99
LowestClassifiedTaxon,,,,,,,,,,,,,,,,,,,,,
A2,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
ASF356,0.0,0.000000,0.0,0.000309,0.001437,0.000088,0.000000,0.001227,0.0,0.000401,...,0.001676,0.0,0.0,0.001359,0.0,0.000574,0.000471,0.000482,0.0,0.0
Acetatifactor,0.0,0.007693,0.0,0.000000,0.000000,0.000000,0.001822,0.000614,0.0,0.000000,...,0.000000,0.0,0.0,0.000056,0.0,0.000000,0.000000,0.000000,0.0,0.0
Acinetobacter,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
Akkermansia-muciniphila,0.0,0.000000,0.0,0.000000,0.000000,0.342675,0.000097,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0


In [4]:
df_count_T = df_count.T
df_count_T.index.name = 'SampleID'

In [5]:
df_count_T.to_excel('16S_relative_abundance.xlsx')